In [2]:
import pandas as pd
import numpy as np
import unicodedata
import re

In [14]:
def harmonise_colonne_df(df: pd.DataFrame, column: str, mapping: dict, other_value=None) -> pd.DataFrame:
    """
    Harmonise les valeurs d'une colonne selon un dictionnaire de correspondance en utilisant replace.
    Les valeurs non listées dans le mapping peuvent être remplacées par other_value.

    Args:
        df          : pd.DataFrame
        column      : nom de la colonne à harmoniser
        mapping     : dictionnaire {valeur à remplacer: valeur cible}
        other_value : valeur pour les valeurs non listées (optionnel)
    
    Returns:
        pd.DataFrame avec la colonne harmonisée
    """
    if column not in df.columns:
        raise ValueError(f"La colonne '{column}' n'existe pas dans le DataFrame.")
    
    # Nettoyer la colonne : convertir en string et enlever espaces
    df[column] = df[column].astype(str).str.strip()
    
    # Remplacer les valeurs selon le mapping
    df[column] = df[column].replace(mapping)
    
    # Remplacer toutes les valeurs qui ne sont pas dans le mapping par other_value
    if other_value is not None:
        df[column] = df[column].where(df[column].isin(mapping.values()), other_value)
    
    return df



In [15]:
def remplacer_valeurs_nulles(df, value_if_numeric=0):
    """
    Remplace les NaN dans un DataFrame selon le type de colonne :
      - numériques : moyenne (si 'mean') ou valeur fixe
      - texte/object : 'Inconnu'
      - datetime : min date existante ou '1970-01-01'
      - bool : False

    Args:
        df (pd.DataFrame): DataFrame à nettoyer
        value_if_numeric (int/float/'mean'): Valeur pour colonnes numériques

    Returns:
        pd.DataFrame: DataFrame nettoyé
    """
    df = df.copy()
    
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], errors='coerce')
            val = df[col].mean() if value_if_numeric == 'mean' else value_if_numeric
            df[col] = df[col].fillna(val)
            df[col] = df[col].astype(int) if pd.api.types.is_integer_dtype(val) else df[col]

        elif pd.api.types.is_datetime64_any_dtype(df[col]):
            min_date = df[col].min() if df[col].notna().any() else pd.Timestamp('1970-01-01')
            df[col] = df[col].fillna(min_date)

        elif pd.api.types.is_bool_dtype(df[col]):
            df[col] = df[col].fillna(False)

        else:
            df[col] = df[col].fillna('Inconnu')
    
    return df


In [ ]:

def standardiser_colonne_caisse(df: pd.DataFrame, column: str, lower=True, strip=True, remove_accents=True) -> pd.DataFrame:
    """
    Standardise une colonne string :
      - met en minuscules (optionnel)
      - enlève les accents (optionnel)
      - enlève les espaces en début/fin (optionnel)
      - remplace les multiples espaces par un seul espace
      - ne garde que les lettres [a-z] et les espaces internes
    
    Args:
        df             : pd.DataFrame
        column         : nom de la colonne à standardiser
        lower          : mettre en minuscules si True
        strip          : enlever les espaces en début/fin si True
        remove_accents : enlever les accents si True
    
    Returns:
        pd.DataFrame avec la colonne standardisée
    """
    if column not in df.columns:
        raise ValueError(f"La colonne '{column}' n'existe pas dans le DataFrame.")
    
    col = df[column].astype(str)
    
    if remove_accents:
        col = col.apply(lambda x: unicodedata.normalize('NFKD', x))
    if lower:
        col = col.str.lower()
    
    if strip:
        col = col.str.strip()
    
    # Remplacer multiples espaces par un seul
    # ici notre fct sub remplace un ou plieseurs(+) espace (\s) par un seule ' '

    col = col.apply(lambda x: re.sub(r'\s+', ' ', x)) 
    
    df[column] = col
    return df


In [ ]:
def traiter_valeurs_speciales(df, valeurs_invalides=None):
    """
    Remplace les valeurs spéciales (-1, '00', 'NULL', etc.) par NaN dans tout le DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame à nettoyer
        valeurs_invalides (list, optional): liste de valeurs à considérer comme invalides
        
    Returns:
        pd.DataFrame: DataFrame avec valeurs spéciales remplacées par NaN
    """
    df = df.copy()
    
    # Valeurs invalides par défaut
    if valeurs_invalides is None:
        valeurs_invalides = ['-1', '00', 'NULL', 'NaN', 'nan', 'None', '']
    
    # Remplacer les valeurs invalides par np.nan
    df = df.replace(valeurs_invalides, np.nan)
    
    return df

In [18]:

def remplacer_non_renseigne(df):
    """
    Remplace dans les colonnes texte toutes les cellules vides, 
    '0', '.' ou ne contenant que des espaces par "Non renseigné" 
    selon la consigne ONISR.
    
    Args:
        df (pd.DataFrame): DataFrame à nettoyer
        
    Returns:
        pd.DataFrame: DataFrame avec les colonnes texte mises à jour
    """
    df = df.copy()
    
    # Valeurs considérées comme non renseignées
    valeurs_non_renseignees = ['0', '.']

    # Parcourir les colonnes de type texte
    for col in df.columns:
        if pd.api.types.is_string_dtype(df[col]) or df[col].dtype == 'object':
            # Supprimer les espaces en début/fin
            df[col] = df[col].astype(str).str.strip()
            # Remplacer les cellules vides ou valeurs spéciales
            df[col] = df[col].replace([''], 'Non renseigné')
            df[col] = df[col].replace(valeurs_non_renseignees, 'Non renseigné')
    
    return df


In [19]:
def remplacer_valeur(df, colonne, ancienne_valeur, nouvelle_valeur):
    """
    Remplace une valeur précise dans une colonne d'un DataFrame.
    
    Args:
        df (pd.DataFrame): le DataFrame à modifier
        colonne (str): le nom de la colonne à traiter
        ancienne_valeur (any): la valeur à remplacer
        nouvelle_valeur (any): la nouvelle valeur
        
    Returns:
        pd.DataFrame: le DataFrame modifié
    """
    df = df.copy()
    df[colonne] = df[colonne].replace(ancienne_valeur, nouvelle_valeur)
    return df


In [20]:

def supprimer_colonnes(df, colonnes_a_supprimer):
    """
    Supprime une ou plusieurs colonnes d'un DataFrame.
    
    Args:
        df (pd.DataFrame): le DataFrame à modifier
        colonnes_a_supprimer (list): liste des noms de colonnes à supprimer
        
    Returns:
        pd.DataFrame: DataFrame avec les colonnes supprimées
    """
    df = df.copy()
    df.drop(columns=colonnes_a_supprimer, inplace=True)
    return df
